# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [22]:

## The following libraries might be useful
!pip install -q langchain-groq
!pip install -U -q langchain-community
!pip install -U -q langchain-chroma
!pip install -U -q rouge_score
!pip install -U -q ragas
!pip install -U -q evaluate
!pip install -U -q langchain-google-genai
!pip install -U -q langchain_huggingface
!pip install faiss-cpu
!pip install -U -q datasets

In [2]:
# Import essential libraries
import warnings
import os
import json
import re
import glob
import random
import logging
from collections import Counter
import numpy as np

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords
from transformers import AutoTokenizer

from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains import RetrievalQA

import chromadb
from chromadb.config import Settings

warnings.filterwarnings('ignore')

nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [3]:
# Load the files as documents
from google.colab import drive
drive.mount('/content/drive/')

from langchain.document_loaders import DirectoryLoader, TextLoader
corpus = "/content/drive/My Drive/RAG LEGAL/rag_legal/corpus"

dict_loader = DirectoryLoader(
    corpus,
    glob="**/*.txt",
    loader_cls = lambda corpus: TextLoader(corpus, encoding="utf-8")
)

docs = dict_loader.load()
print(f"Loaded {len(docs)} documents")

Mounted at /content/drive/
Loaded 698 documents


#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [4]:
# Clean and preprocess the data
st_word = set(stopwords.words('english'))

def preprocess(txt):
    txt = txt.lower()
    txt = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w{2,4}\b', '', txt)
    txt = re.sub(r'\+?\d[\d\s().-]{7,}', '', txt)
    txt = re.sub(r'^\s*\d+(?:\.\d+)*\.\s+', '', txt, flags=re.MULTILINE)
    txt = re.sub(r'[^a-z0-9\s]', ' ', txt)
    txt = re.sub(r'\s+', ' ', txt).strip()
    txt = ' '.join([word for word in txt.split() if word not in st_word])
    return txt

cl_docs = []

for i, doc in enumerate(docs):
    try:
        if not doc.page_content or not isinstance(doc.page_content, str):
            raise ValueError("Missing Content")
        clean_txt = preprocess(doc.page_content)
        clean_doc = Document(page_content=clean_txt, metadata=doc.metadata)
        cl_docs.append(clean_doc)

    except Exception as exception:
        print(f"skipping document {i} ({doc.metadata.get('source', 'unknown')}): {exception}")

### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [5]:
# Calculate the average, maximum and minimum document length.
counts = [len(doc.page_content.split()) for doc in cl_docs]
avg_len = sum(counts) / len(counts)
print("Average length of document", avg_len)
max_len = max(counts)
print("Maximum length of document", max_len)
min_len = min(counts)
print("Minimum length of document", min_len)

Average length of document 9161.199140401146
Maximum length of document 86518
Minimum length of document 146


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [6]:
# Find frequency of occurence of words
from collections import Counter
clean_text = " ".join(doc.page_content for doc in cl_docs)
tk = clean_text.split()
freq = Counter(tk)

# Top 20 most common words
print("Top 20 most common words in the text:")
for word, count in freq.most_common(20):
    print(f"{word}: {count}")
print()
# 20 least common words (appearing more than once)
least_common = [(w, c) for w, c in freq.items() if c > 1]
least_common = sorted(least_common, key=lambda x: x[1])[:20]
print("20 least common words (count > 1) in the text:")
for word, count in least_common:
    print(f"{word}: {count}")

Top 20 most common words in the text:
company: 156422
shall: 108015
agreement: 104655
section: 75412
parent: 60715
party: 54208
date: 39392
time: 35827
1: 35299
material: 34242
merger: 33907
b: 33656
subsidiaries: 33320
applicable: 31384
3: 30806
2: 30530
including: 29406
respect: 28849
may: 28108
stock: 26887

20 least common words (count > 1) in the text:
fullstory: 2
knights: 2
wroclaw: 2
pleas: 2
decompiling: 2
dll: 2
sente: 2
soient: 2
anglais: 2
nw10: 2
7pa: 2
shape: 2
invokes: 2
tent: 2
5005: 2
mcdowell: 2
dat: 2
kerber: 2
stoney: 2
logically: 2


#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [7]:
# Transform the page contents of documents

# Compute similarity scores
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
texts = [doc.page_content for doc in cl_docs]
vector = TfidfVectorizer()
tfidf_matrix = vector.fit_transform(texts)
simi_matrix = cosine_similarity(tfidf_matrix)
simi_df = pd.DataFrame(simi_matrix)

print("Similarity between first 10 documents:")
print(simi_df.iloc[:10, :10])

Similarity between first 10 documents:
          0         1         2         3         4         5         6  \
0  1.000000  0.451724  0.323819  0.391353  0.345546  0.187040  0.349000   
1  0.451724  1.000000  0.694658  0.599945  0.649519  0.394753  0.664968   
2  0.323819  0.694658  1.000000  0.497617  0.760569  0.489618  0.760492   
3  0.391353  0.599945  0.497617  1.000000  0.489675  0.288887  0.490382   
4  0.345546  0.649519  0.760569  0.489675  1.000000  0.440655  0.726113   
5  0.187040  0.394753  0.489618  0.288887  0.440655  1.000000  0.451181   
6  0.349000  0.664968  0.760492  0.490382  0.726113  0.451181  1.000000   
7  0.119671  0.163394  0.147407  0.132295  0.166708  0.089094  0.169192   
8  0.264017  0.332271  0.313704  0.271126  0.363180  0.189095  0.352920   
9  0.432005  0.852015  0.694280  0.572303  0.665718  0.386420  0.652322   

          7         8         9  
0  0.119671  0.264017  0.432005  
1  0.163394  0.332271  0.852015  
2  0.147407  0.313704  0.694280  

In [8]:
# create a list of 10 random integers
random_indices = random.sample(range(len(cl_docs)), 10)
# Display the randomly selected documents
for i in random_indices:
    print(f"Document {i} (source: {cl_docs[i].metadata.get('source', 'unknown')}):")

Document 677 (source: /content/drive/My Drive/RAG LEGAL/rag_legal/corpus/maud/Alaska Communications Systems Group, Inc._Investment Group.txt):
Document 502 (source: /content/drive/My Drive/RAG LEGAL/rag_legal/corpus/cuad/SouthernStarEnergyInc_20051202_SB-2A_EX-9_801890_EX-9_Affiliate Agreement.txt):
Document 682 (source: /content/drive/My Drive/RAG LEGAL/rag_legal/corpus/maud/New_Home_Co_Apollo_Global_Management.txt):
Document 125 (source: /content/drive/My Drive/RAG LEGAL/rag_legal/corpus/cuad/WPPPLC_04_30_2020-EX-4.28-SERVICE AGREEMENT.txt):
Document 671 (source: /content/drive/My Drive/RAG LEGAL/rag_legal/corpus/maud/Mackinac Financial Corporation_Nicolet Bankshares, Inc..txt):
Document 431 (source: /content/drive/My Drive/RAG LEGAL/rag_legal/corpus/cuad/BLACKBOXSTOCKSINC_08_05_2014-EX-10.1-DISTRIBUTOR AGREEMENT.txt):
Document 110 (source: /content/drive/My Drive/RAG LEGAL/rag_legal/corpus/cuad/NICELTD_06_26_2003-EX-4.5-OUTSOURCING AGREEMENT.txt):
Document 306 (source: /content/driv

In [9]:
# Compute similarity scores for 10 random documents
random_docs = [cl_docs[i].page_content for i in random_indices]
tfidf_matrix_r = vector.transform(random_docs)
simi_matrix_r = cosine_similarity(tfidf_matrix_r)
simi_df_r = pd.DataFrame(simi_matrix_r)
print("Similarity for randomly selected documents:")
print(simi_df_r)

Similarity for randomly selected documents:
          0         1         2         3         4         5         6  \
0  1.000000  0.072054  0.931337  0.432266  0.327331  0.339393  0.069016   
1  0.072054  1.000000  0.068626  0.060971  0.030718  0.066269  0.029978   
2  0.931337  0.068626  1.000000  0.453467  0.310728  0.347206  0.068034   
3  0.432266  0.060971  0.453467  1.000000  0.185053  0.245189  0.044944   
4  0.327331  0.030718  0.310728  0.185053  1.000000  0.139784  0.027484   
5  0.339393  0.066269  0.347206  0.245189  0.139784  1.000000  0.076195   
6  0.069016  0.029978  0.068034  0.044944  0.027484  0.076195  1.000000   
7  0.057201  0.021847  0.053362  0.034984  0.024511  0.057784  0.020462   
8  0.083630  0.032829  0.079526  0.048803  0.035610  0.094974  0.031980   
9  0.065059  0.025505  0.062326  0.037999  0.033639  0.052596  0.024157   

          7         8         9  
0  0.057201  0.083630  0.065059  
1  0.021847  0.032829  0.025505  
2  0.053362  0.079526  0.062

### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [10]:
# Process files and generate chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
documents = text_splitter.split_documents(doc for doc in docs)
ch_num = len(documents)
print("Chunks created: ", ch_num)

Chunks created:  91792


## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [12]:
# Fetch your OPENAI API Key as an environment variable
# Using GOOGLE_API_KEY
import os
from dotenv import load_dotenv
dotenv_path = '/content/s.env'
load_dotenv(dotenv_path=dotenv_path)
gem_api = os.getenv("GOOGLE_API_KEY")

In [13]:
# Initialise an embedding function
from langchain_huggingface import HuggingFaceEmbeddings
embed = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-MiniLM-L6-v2" )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [14]:
# Add Chunks to vector DB
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embed)
total = db.index.ntotal
print("Number of documents: ", db.index.ntotal)

Number of documents:  91792


### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [15]:
# Create a RAG chain
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key = gem_api , temperature=0)
retriever = db.as_retriever(search_kwargs={"k": 4})
rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [16]:
# Create a function for question answering
def ans_ques(question):
    result = rag.invoke({"query": question})
    answer = result['result']
    source_docs = result['source_documents']

    return answer, source_docs

In [17]:
# Example question
question ="Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"
print(ans_ques(question)[0],": ",ans_ques(question)[1])

The provided text does not explicitly state that the agreement does *not* grant the receiving party any rights to the confidential information.  However, it heavily restricts the use and disclosure of confidential information, implying that the rights granted are limited to those specifically outlined in the agreement. :  [Document(id='058a2b60-5536-4898-bf82-164d9fecc335', metadata={'source': '/content/drive/My Drive/RAG LEGAL/rag_legal/corpus/contractnli/simply-fashion---standard-nda.txt'}, page_content='CONFIDENTIALITY AND NON-DISCLOSURE AGREEMENT'), Document(id='35b598d2-1ba5-445a-bae4-391c88c143e1', metadata={'source': '/content/drive/My Drive/RAG LEGAL/rag_legal/corpus/cuad/REGANHOLDINGCORP_03_31_2008-EX-10-LICENSE AND HOSTING AGREEMENT.txt'}, page_content="Disclosure. The Receiving Party may disclose Confidential Information only to its own officers, directors, and employees and to its consultants, subcontractors, and advisors who reasonably need to know it for the purposes cont

## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [18]:
# Create a question set by taking all the questions from the benchmark data
# Also create a ground truth/answer set

import os
import json
import glob
benchmark = "/content/drive/My Drive/RAG LEGAL/rag_legal/benchmarks"
benchmark_files = glob.glob(os.path.join(benchmark, "*.json"))

questions = []
ground_truths = []
for file_path in benchmark_files:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for test in data.get("tests", []):
                if "query" in test:
                    questions.append(test["query"])
                combined_answer = " ".join([snippet.get("answer", "") for snippet in test.get("snippets", [])])
                ground_truths.append(combined_answer)

    except json.JSONDecodeError:
        print("Error in JSON file")
    except Exception:
        print("ERROR")

if questions:
    print("Sample Question:", questions[0])
    print("Sample Ground Truth:", ground_truths[0])



Sample Question: Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
Sample Ground Truth: Any and all proprietary rights, including but not limited to rights to and in inventions, patent rights, utility models, copyrights, trademarks and trade secrets, in and to any Confidential Information shall be and remain with the Participants respectively, and Mentor shall not have any right, license, title or interest in or to any Confidential Information, except the limited right to review, assess and help develop such Confidential Information in connection with the Copernicus Accelerator 2017.


#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [19]:
# Function to evaluate the RAG pipeline

import evaluate
from ragas import evaluate as ragas_evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
from datasets import Dataset
rouge_metric = evaluate.load('rouge')
bleu_metric = evaluate.load('bleu')

def evaluate_rag_pipeline(generated_answers, ground_truths, questions_list, contexts_list):
    rouge_results = rouge_metric.compute(predictions=generated_answers, references=ground_truths)
    bleu_results = bleu_metric.compute(predictions=generated_answers, references=[[gt] for gt in ground_truths])
    ragas_data = {
        "question": questions_list,
        "answer": generated_answers,
        "contexts": [[doc.page_content for doc in context_docs] for context_docs in contexts_list],
        "ground_truth": ground_truths
    }
    ragas_dataset = Dataset.from_dict(ragas_data)
    ragas_results = ragas_evaluate(
        ragas_dataset,
        metrics=[
            faithfulness,
            answer_relevancy,
            context_recall,
            context_precision,
        ],
        llm=llm,
        embeddings=embed,
    )
    final_results = {
        "rouge": rouge_results,
        "bleu": bleu_results,
        "ragas": ragas_results
    }

    return final_results

#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [21]:
# Evaluate the RAG pipeline
from tqdm import tqdm
import pandas as pd
num_questions_to_evaluate = 100
if len(questions) < num_questions_to_evaluate:
    num_questions_to_evaluate = len(questions)

eval_questions = questions[:num_questions_to_evaluate]
eval_ground_truths = ground_truths[:num_questions_to_evaluate]
generated_answers = []
retrieved_contexts = []

for question in tqdm(eval_questions):
    try:
        answer, source_docs = ans_ques(question)
        generated_answers.append(answer)
        retrieved_contexts.append(source_docs)
    except Exception:
        print(f"Error answering")
        generated_answers.append("ERROR - GENERATION FAILED")
        retrieved_contexts.append([])

if len(generated_answers) == len(eval_questions):
    evaluation_results = evaluate_rag_pipeline(
        generated_answers=generated_answers,
        ground_truths=eval_ground_truths,
        questions_list=eval_questions,
        contexts_list=retrieved_contexts
    )

    print("ROUGE-L & BLEU Score")
    print(f"ROUGE-L Score: {evaluation_results['rouge']['rougeL']:.4f}")
    print(f"BLEU Score:    {evaluation_results['bleu']['bleu']:.4f}")

    print("Ragas Score")
    ragas_df = evaluation_results['ragas'].to_pandas()
    print(f"Faithfulness:      {ragas_df['faithfulness'].mean():.4f}")
    print(f"Answer Relevancy:  {ragas_df['answer_relevancy'].mean():.4f}")
    print(f"Context Recall:    {ragas_df['context_recall'].mean():.4f}")
    print(f"Context Precision: {ragas_df['context_precision'].mean():.4f}")

else:
    print("Error")

100%|██████████| 100/100 [01:38<00:00,  1.01it/s]


Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

ROUGE-L & BLEU Score
ROUGE-L Score: 0.1995
BLEU Score:    0.0541
Ragas Score
Faithfulness:      0.9154
Answer Relevancy:  0.6901
Context Recall:    0.5190
Context Precision: 0.4736


## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.

**DATA INSIGHTS:**

The project effectively created a Retrieval-Augmented Generation (RAG) pipeline for legal document analysis. The dataset, including varied legal agreements, was carefully preprocessed through noise removal and text normalization, providing a clean input for further processing. Preliminary data exploration indicated differing document lengths and determined common legal terminology, which guided the chunking approach.


**MODEL PIPELINE:**

The RAG pipeline successfully chunked documents into smaller pieces, which were then embedded with `sentence-transformers/all-MiniLM-L6-v2` and saved to a FAISS vector database. A `RetrievalQA` chain, using `gemini-1.5-flash`, was set up to retrieve pertinent document chunks and produce answers.


**FINAL RESULTS:**

Evaluation results gave important insights. The low ROUGE-L (0.1995) and BLEU (0.0541) values mean that even though the system produces answers, their linguistic relevance and proximity to ground truth answers can be greatly enhanced. Yet, the high Faithfulness score value (0.9154) is a positive indicator that the produced responses are adequately grounded on the retrieved context and minimize hallucinations. The moderate Answer Relevancy (0.6901) reveals improvement potential in making answers more specifically fit the question. Most importantly, the lower Context Recall (0.5190) and Context Precision (0.4736) point to the main area of optimization: the retrieval mechanism must be improved in order to retrieve all pertinent information and filter out irrelevant chunks consistently, which would, in turn, improve overall answer quality.
